In [1]:

import random                                                              
                                                                           
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.display import display, HTML
import seaborn as sns
import time

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler

In [2]:
ratings_cleaned=pd.read_csv(r'data/processed/ratings_cleaned.csv')
books_cleaned = pd.read_csv(r'data/processed/books_cleaned.csv')

In [3]:
# print total number of users and total books we are looking at
num_users = ratings_cleaned.user_id.unique().shape[0]
num_books = ratings_cleaned.book_id.unique().shape[0]
print("Total Users: " + str(
    num_users) + " | Total Books: " + str(num_books))

# sparsity of data
sparsity = round(1.0-len(ratings_cleaned)/float(num_users*num_books),3)
print("Sparsity: " + str(sparsity*100) + "%")

Total Users: 16027 | Total Books: 9999
Sparsity: 99.5%


In [4]:
# convert ratings to pivot table and fills nan with 0
pivot_ratings = ratings_cleaned.pivot(index='user_id', columns='book_id', values='rating')
pivot_ratings = pivot_ratings.fillna(0)#.apply(lambda row: row.fillna(0)

# check which books were not rated and add them in as a column of 0s
# book_id_not = []
# book_id_possible = np.arange(1,10001)

# for i in book_id_possible:
#     if i not in ratings_cleaned['book_id'].unique():
#         book_id_not.append(i)

# display(book_id_not)
#master_table.insert(loc=8616, column=8616, value=0)
pivot_ratings.insert(loc=8882, column=8882, value=0)

pivot_ratings

MemoryError: Unable to allocate 1.19 GiB for an array with shape (16027, 9999) and data type float64

In [ ]:
pivot_ratings_knn = csr_matrix(pivot_ratings.T.values)
knn = NearestNeighbors(metric='cosine',algorithm='brute')
knn.fit(pivot_ratings_knn)

In [ ]:
# convert book name to book_id
book_id = books_cleaned[books_cleaned['title']=='Twilight (Twilight, #1)']['book_id'].values[0]
# find the 10 most similar books based on cosine similarity
book_distances, book_indices = knn.kneighbors(pivot_ratings.T.loc[book_id,:].values.reshape(1,-1),n_neighbors=10)

display(book_distances)
book_indices

In [ ]:
for i in range(0,len(book_distances.flatten())):
        if i ==0:
            print('Books similar to book {0}:\n'.format('Twilight (Twilight, #1)')) #item_master_table.index[user_id]
        else:
            print('{0}: {1}, with distance of {2}'.format(i,books_cleaned[books_cleaned['book_id']==(book_indices.flatten()[i]+1)]['title'].values[0], book_distances.flatten()[i]))

In [ ]:
def item_item_cf(book_title, num_books):
    # gets top 9 similar books and similarity score
    # converts book title to book_id
    book_id = books_cleaned[books_cleaned['title']==book_title]['book_id'].values
    book_distances,book_indices = knn.kneighbors(pivot_ratings.T.loc[book_id,:].values.reshape(1,-1),n_neighbors=num_books+1)
    for i in range(0,len(book_distances.flatten())):
        if i ==0:
            print('')
            print('Books similar to book {0}:\n'.format(book_title))
        else:
            print('{0}: {1}, with distance of {2}'.format(i,books_cleaned[books_cleaned['book_id']==(book_indices.flatten()[i]+1)]['title'].values[0],book_distances.flatten()[i]))

In [ ]:
item_item_cf('Twilight (Twilight, #1)', 10)

In [ ]:
def item_based_cf(user_id, threshold_rating, num_books):
    find_books_similar_to = ratings_cleaned[(ratings_cleaned['user_id'] == user_id) & (ratings_cleaned['rating'] >= threshold_rating)]['title'].array
    for book in find_books_similar_to:
        item_item_cf(book, num_books)
    print('i')

In [ ]:
item_based_cf(7,5,2)

In [ ]:
## User-Based Collaborative Filtering

In [ ]:
ratings_cleaned[ratings_cleaned['user_id']==19]
pivot_ratings_knn = csr_matrix(pivot_ratings.values)
knn = NearestNeighbors(metric='cosine',algorithm='brute')
knn.fit(pivot_ratings_knn)

user_distances,user_indices = knn.kneighbors(pivot_ratings.loc[19,:].values.reshape(1,-1),n_neighbors=30)
similar_users = pivot_ratings.index[user_indices.flatten()]

for i in range(0,10):
    if i ==0:
            # if user is user given
        print('Users similar to user {0}:\n'.format('19')) #master_table.index[user_id]
    else:
            # else print top 9 users 
            # gets location top users and prints their user_id
        print('{0}: {1}, with distance of {2}'.format(i,pivot_ratings.index[user_indices.flatten()[i]],user_distances.flatten()[i]))
      

In [ ]:
user_based_cf_df = ratings_cleaned.groupby('user_id').agg(np.mean).drop(['book_id'],axis=1)

user_based_cf_df = ratings_cleaned.merge(user_based_cf_df, how='inner',on='user_id')
user_based_cf_df['mean_difference'] = user_based_cf_df['rating_x'] - user_based_cf_df['rating_y']
user_based_cf_df

# rating_x : rating given by user_id to book_id
# rating_y : average rating of user_id
# mean_difference: rating_x - rating_y

In [ ]:
pivot_mean_difference = pd.pivot_table(user_based_cf_df,values='mean_difference',index='user_id',columns='book_id')
pivot_mean_difference = pivot_mean_difference.fillna(0) #user_master_table.mean(axis=0))
pivot_mean_difference.insert(loc=8882, column=8882, value=0)
pivot_mean_difference

In [ ]:
similarity_scores = sum(1 - user_distances.flatten()) # denominator
similarity_scores_T = 1-user_distances.T # similarity scores for each user
# putting the equation together
pivot_mean_difference = pivot_mean_difference.loc[similar_users,:].multiply(similarity_scores_T/similarity_scores, axis=0)
# entire fraction summation
pivot_mean_difference.loc['Column_Total'] = pivot_mean_difference.sum(axis=0) 

# user 19's average rating
u_avg_rating = ratings_cleaned.groupby('user_id').agg(np.mean).loc[19,'rating']# 3.2083333333333335

# adding user 19's average rating to each user-book pair score and sorting them from highest to lowest
recs = sorted(list(enumerate(pivot_mean_difference.loc['Column_Total'] + u_avg_rating, 1)),key=lambda x: x[1],reverse=True)

In [ ]:
# convert list into dataframe only showing top 10 books
predicted_books = pd.DataFrame(recs,columns=['book_id','predicted_rating'])[:10]
display(predicted_books.head())


# dataframe showing how similar users rated the books recommended
display(user_based_cf_df[(user_based_cf_df['user_id'].isin(similar_users)) & (user_based_cf_df['book_id'].isin(predicted_books['book_id']))].sort_values('book_id'))

In [ ]:
predicted_books['book_id'][0]

In [ ]:
def user_based_cf(user_id,k_neighbors, n_books):
    '''
        Finds the closest neighbors (k_neighbors) in terms of cosine similarity for a given user (user_id) and
        Returns the top books (n_books) the model predicts the user will rate highly 
        
    '''

    user_distances,user_indices = knn.kneighbors(pivot_ratings.loc[user_id,:].values.reshape(1,-1),n_neighbors=k_neighbors)
    similar_users = pivot_ratings.index[user_indices.flatten()]

    for i in range(0,len(user_distances.flatten())):
        if i ==0:
                # if user is user given
            print('Users similar to user {0}:\n'.format(user_id)) #master_table.index[user_id]
        else:
                # else print top 9 users 
                # gets location top users and prints their user_id
            print('{0}: {1}, with distance of {2}'.format(i,pivot_ratings.index[user_indices.flatten()[i]],user_distances.flatten()[i]))

    user_based_cf_df = ratings_cleaned.groupby('user_id').agg(np.mean).drop(['book_id'],axis=1)
    user_based_cf_df = user_based_cf_df.merge(ratings_cleaned, how='inner',on='user_id')
    user_based_cf_df['mean_difference'] = user_based_cf_df['rating_x'] - user_based_cf_df['rating_y']

    similarity_scores = sum(1 - user_distances.flatten()) # denominator
    similarity_scores_T = 1-user_distances.T # similarity scores for each user
    # putting the equation together
    pivot_mean_difference_func = pivot_mean_difference.loc[similar_users,:].multiply(similarity_scores_T/similarity_scores, axis=0)
    # entire fraction summation
    pivot_mean_difference_func.loc['Column_Total'] = pivot_mean_difference_func.sum(axis=0) 

    # user's average rating
    u_avg_rating = ratings_cleaned.groupby('user_id').agg(np.mean).loc[user_id,'rating']# 3.2083333333333335

    # adding user's average rating to each user-book pair score and sorting them from highest to lowest
    recs = sorted(list(enumerate(pivot_mean_difference_func.loc['Column_Total'] + u_avg_rating, 1)),key=lambda x: x[1],reverse=True)
    # recs in a dataframe
    recs_df = pd.DataFrame(recs,columns=['book_id','predicted_rating'])[:n_books]
    
    for i in range(0,len(recs_df)):
        if i ==0:
                # if user is user given
            print('')
            print('Top {0} Books liked by {0} people most similar to user 19:\n'.format(n_books, k_neighbors)) #master_table.index[user_id]
        else:
                # else print top 9 users 
                # gets location top users and prints their user_id
            print('{0}: {1}, with your predicted rating of {2}'.format(i,recs_df['book_id'][i],recs_df['predicted_rating'][i]))

    

In [ ]:
user_based_cf(19,30,10)

## Model-Based Collaborative Filtering

In [ ]:
# 80% trainset, 20% testset  
shuffled = pivot_ratings.replace(0,np.NaN).sample(frac = 1,axis=0,random_state=42)
threshold = int(.8 * len(shuffled))                                     
train = shuffled[:threshold]                             
test = shuffled[threshold:]
train_indices = train.index.values
test_indices = test.index.values

# normalize ratings
train_means = np.nan_to_num(np.mean(train,axis=1).values)
train_demeaned = train.sub(train_means,axis=0).replace(np.NaN,0)

In [ ]:
# get 10 SVD components from train matrix
start10 = time.time()
u,s,vt = svds(train_demeaned,k=10)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
svd_10_rmse = mean_squared_error(X_pred,train_demeaned,squared=True)
end10 = time.time()
print('SVD Using 10 Features RMSE: ' + str(svd_10_rmse))
print(end10 - start10)

# get 20 SVD components from train matrix
start20 = time.time()
u,s,vt = svds(train_demeaned,k=20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
svd_20_rmse = mean_squared_error(X_pred,train_demeaned,squared=True)
end20 = time.time()
print('SVD Using 20 Features RMSE: ' + str(svd_20_rmse))
print(end20 - start20)

# get 50 SVD components from train matrix
start50 = time.time()
u,s,vt = svds(train_demeaned,k=50)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
svd_50_rmse = mean_squared_error(X_pred,train_demeaned,squared=True)
end50 = time.time()
print('SVD Using 50 Features RMSE: ' + str(svd_50_rmse))
print(end50 - start50)

# get 100 SVD components from train matrix
start100 = time.time()
u,s,vt = svds(train_demeaned,k=100)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
svd_100_rmse = mean_squared_error(X_pred,train_demeaned,squared=True)
end100 = time.time()
print('SVD Using 100 Features RMSE: ' + str(svd_100_rmse))
print(end100 - start100)

In [ ]:
# create user-book pivot table with weighted ratings as values
pivot_weighted_ratings = books_weighted_ranking.merge(ratings_cleaned, how='inner',on='book_id').pivot(index='user_id', columns='book_id', values='weighted_score').fillna(0)
# insert book that wasn't rated
pivot_weighted_ratings.insert(loc=8882, column=8882, value=0)

# split data
# weighted trainset is the same as svd trainset
weightedshuffled = pivot_weighted_ratings.loc[train_indices,:]
start_weighted = time.time()
weightedpopular_rmse = mean_squared_error(weightedshuffled,train_demeaned,squared=True)
end_weighted = time.time()
print(end_weighted - start_weighted)
print('Weight-Based RMSE: ' + str(weightedpopular_rmse))

In [ ]:
rmse = [svd_10_rmse,svd_20_rmse,svd_50_rmse,svd_100_rmse, weightedpopular_rmse]
models = ['Train SVD with 10 Features','Train SVD 20 Features','Train SVD 50 Features','Train SVD 100 Features', 'Train Weighted Average']
plt.bar(models, rmse);
plt.xticks(rotation=90);
plt.ylabel('RMSE');
plt.title('RMSE for Different Collaborative Filtering Models');

In [ ]:
# normalize ratings
test_nan = np.nan_to_num(np.mean(test,axis=1).values)
test_demeaned = test.sub(test_nan,axis=0).replace(np.NaN,0)


# get 10 SVD components from train matrix
starttest = time.time()
u,s,vt = svds(test_demeaned,k=10)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
test_svd_10_rmse = mean_squared_error(X_pred,test_demeaned,squared=True)
endtest = time.time()
print('Test RMSE Using 10 Features: ' + str(test_svd_10_rmse))
print(endtest - starttest)

In [ ]:
rmse = [test_svd_10_rmse,svd_10_rmse,svd_20_rmse,svd_50_rmse,svd_100_rmse, weightedpopular_rmse]
models = ['Test SVD with 10 Features','Train SVD with 10 Features','Train SVD 20 Features','Train SVD 50 Features','Train SVD 100 Features', 'Train Weighted Average']
plt.bar(models, rmse);
plt.xticks(rotation=90);
plt.ylabel('RMSE');
plt.title('RMSE for Different Collaborative Filtering Models');

In [ ]:
pivot_ratings_nan = pivot_ratings.replace(0,np.NaN)


# get means for each user
user_means = np.nan_to_num(np.mean(pivot_ratings_nan,axis=1).values)
# subtract means from dataset and replace not-yet-rated items with 0
all_demeaned = pivot_ratings_nan.sub(user_means,axis=0).replace(np.NaN,0)

# get 10 SVD components from train matrix
start10 = time.time()
u,s,vt = svds(all_demeaned,k=10)
s_diag_matrix=np.diag(s)
X_pred_all = np.dot(np.dot(u, s_diag_matrix), vt)
all_svd_10_rmse = mean_squared_error(X_pred_all,all_demeaned,squared=True)
end10 = time.time()
all_svd_10_rmse

In [ ]:
# Transform features by scaling each feature from 1 to 5
sc=MinMaxScaler(feature_range = (1,5))
svd_mat = sc.fit_transform(X_pred_all.T + user_means)

# convert scaled matrix to df
svd_df = pd.DataFrame(svd_mat.T, columns=pivot_ratings_nan.columns,index=pivot_ratings_nan.index)
svd_df

In [ ]:
def svd_rec(user_id,num_recs):
    print('Books user {0} has highly rated read:'.format(user_id))
    print(user_based_cf_df[(user_based_cf_df['user_id'] == user_id) & (user_based_cf_df['rating_x'] > 3)][['title', 'rating_x','rating_y']])
   
    # dataframe of all books and predicted scores, sorted
    user_recs = pd.DataFrame(svd_df.loc[user_id].sort_values(ascending=False).reset_index()).rename(columns = {user_id : 'predicted_rating'})
    # find books that user has already read
    user_read = user_based_cf_df[user_based_cf_df['user_id'] == user_id]
    # take away books the user has read
    # df: book_id | predicted_rating | title
    new_books = user_recs[~user_recs['book_id'].isin(user_read)][:num_recs].merge(ratings_cleaned[['book_id','title']],on='book_id',how='inner').drop_duplicates().reset_index()
    new_books
    
    # recommend the top n books with their titles and predicted rating
    print('')
    print('Top {0} Books we think user {1} would rate the highest:\n'.format(num_recs, user_id))

    for i in range(0,len(new_books)):
        print('{0}: {1}, with your predicted rating of {2}'.format(i+1,new_books['title'][i],new_books['predicted_rating'][i]))


def predict_rating(user_id,book_title):
    # we will use SVD with 10 features
    book_id = books_cleaned[books_cleaned['title']==book_title]['book_id'].values
    print('The predicted rating for user {0} for {1} is {2}'.format(user_id,book_title,svd_df.loc[user_id,book_id])) 

In [ ]:
svd_rec(10560,10)

In [ ]:
predict_rating(52414,'Harry Potter and the Half-Blood Prince (Harry Potter, #6)')